In [2]:
import pandas as pd
from Bio import SeqIO

In [3]:
blast = pd.read_csv('/home/collot/stage/collot/collot/out_ortho/hs_blast/out/A1BG_9606.out', sep='\t', header=None)
blast.columns = ["query", "subject", "identity", "alignment_length", "mismatches", "gap_opens","q_start", "q_end", "s_start", "s_end", "evalue", "bit_score"]
mask = blast['query'] == blast['subject']
blast = blast[~mask] 
blast['query_specie'] = blast['query'].apply(lambda x: x.split('_')[-1])
blast['subject_specie'] = blast['subject'].apply(lambda x: x.split('_')[-1])
print(blast)

                           query                        subject  identity  \
1        A1BG_NM_130786.4-8_9606    A1BG_XM_030819447.1-7_61853    97.059   
2        A1BG_NM_130786.4-8_9606     A1BG_XM_001146669.7-8_9598    99.432   
3        A1BG_NM_130786.4-8_9606     A1BG_XM_055103953.3-8_9597    99.432   
4        A1BG_NM_130786.4-8_9606     A1BG_XM_031004187.3-8_9595    98.864   
5        A1BG_NM_130786.4-8_9606     A1BG_XM_055239924.2-8_9590    98.295   
...                          ...                            ...       ...   
4788  A1BG_XM_001146669.7-1_9598        A1BG_NM_130786.4-1_9606   100.000   
4789  A1BG_XM_001146669.7-1_9598     A1BG_XM_054464970.2-1_9600    95.556   
4790  A1BG_XM_001146669.7-1_9598     A1BG_XM_055239924.2-1_9590    95.556   
4791  A1BG_XM_001146669.7-1_9598  A1BG_XM_017509623.2-1_2715852    90.099   
4792  A1BG_XM_001146669.7-1_9598     A1BG_XM_031004187.3-1_9595    98.214   

      alignment_length  mismatches  gap_opens  q_start  q_end  s_start  s_e

In [4]:
#check if there is more than one hit for each specie
for exons in blast['query'].unique():
    df_exon = blast[blast['query']==exons]
    df_exon = df_exon.sort_values(by=['subject_specie'])
    if df_exon.duplicated(subset='subject_specie').sum() >0 : 
        print(df_exon.duplicated(subset='subject_specie').sum())
        print(df_exon)

3
                      query                        subject  identity  \
71  A1BG_NM_130786.4-6_9606  A1BG_XM_017509623.2-5_2715852    90.234   
76  A1BG_NM_130786.4-6_9606    A1BG_XM_012758098.2-6_30608    83.857   
69  A1BG_NM_130786.4-6_9606    A1BG_XM_012473147.3-6_37293    91.406   
74  A1BG_NM_130786.4-6_9606   A1BG_XM_012655895.1-7_379532    84.000   
73  A1BG_NM_130786.4-6_9606    A1BG_XM_039466939.1-7_39432    89.453   
57  A1BG_NM_130786.4-6_9606    A1BG_XM_033224719.1-6_54180    95.200   
63  A1BG_NM_130786.4-6_9606   A1BG_XM_023197509.3-6_591936    94.141   
61  A1BG_NM_130786.4-6_9606    A1BG_XM_038006460.1-4_60711    94.800   
62  A1BG_NM_130786.4-6_9606    A1BG_XM_017850680.1-6_61621    94.800   
60  A1BG_NM_130786.4-6_9606    A1BG_XM_010386959.2-6_61622    94.800   
46  A1BG_NM_130786.4-6_9606    A1BG_XM_030819447.1-5_61853    97.872   
54  A1BG_NM_130786.4-6_9606    A1BG_XM_032762288.2-6_81572    97.518   
75  A1BG_NM_130786.4-6_9606   A1BG_XM_069458507.1-6_859984    

In [5]:
#reciprocal hits 
reciprocal_hits=[]
for index, row in blast.iterrows():
    que = row['query']
    sub = row['subject']
    if que in blast['subject'].values:
        match = blast[blast['subject']==que]
        if sub in match['query'].values:
            reciprocal_hits.append(row)
df_reci= pd.DataFrame(reciprocal_hits)
print(df_reci)
if len(df_reci)==len(blast):
    print("all hits are reciprocal")

                           query                        subject  identity  \
1        A1BG_NM_130786.4-8_9606    A1BG_XM_030819447.1-7_61853    97.059   
2        A1BG_NM_130786.4-8_9606     A1BG_XM_001146669.7-8_9598    99.432   
3        A1BG_NM_130786.4-8_9606     A1BG_XM_055103953.3-8_9597    99.432   
4        A1BG_NM_130786.4-8_9606     A1BG_XM_031004187.3-8_9595    98.864   
5        A1BG_NM_130786.4-8_9606     A1BG_XM_055239924.2-8_9590    98.295   
...                          ...                            ...       ...   
4788  A1BG_XM_001146669.7-1_9598        A1BG_NM_130786.4-1_9606   100.000   
4789  A1BG_XM_001146669.7-1_9598     A1BG_XM_054464970.2-1_9600    95.556   
4790  A1BG_XM_001146669.7-1_9598     A1BG_XM_055239924.2-1_9590    95.556   
4791  A1BG_XM_001146669.7-1_9598  A1BG_XM_017509623.2-1_2715852    90.099   
4792  A1BG_XM_001146669.7-1_9598     A1BG_XM_031004187.3-1_9595    98.214   

      alignment_length  mismatches  gap_opens  q_start  q_end  s_start  s_e

In [9]:
#add query length 
exon_length = {}
with open("/home/collot/stage/collot/collot/out_ortho/fasta_homo_sapiens/A1BG_9606.fasta", "r") as f:
    for record in SeqIO.parse(f, "fasta"):
        exon = record.description
        exon_length[exon]=len(record.seq)


ali_length = []
for e in df_reci['query'].values:
    if e in exon_length:
        ali_length.append(exon_length[e])
df_reci['query_length']=ali_length
df_reci['coverage']= (df_reci['alignment_length']/df_reci['query_length'])*100
print(df_reci)

                           query                        subject  identity  \
1        A1BG_NM_130786.4-8_9606    A1BG_XM_030819447.1-7_61853    97.059   
2        A1BG_NM_130786.4-8_9606     A1BG_XM_001146669.7-8_9598    99.432   
3        A1BG_NM_130786.4-8_9606     A1BG_XM_055103953.3-8_9597    99.432   
4        A1BG_NM_130786.4-8_9606     A1BG_XM_031004187.3-8_9595    98.864   
5        A1BG_NM_130786.4-8_9606     A1BG_XM_055239924.2-8_9590    98.295   
...                          ...                            ...       ...   
4788  A1BG_XM_001146669.7-1_9598        A1BG_NM_130786.4-1_9606   100.000   
4789  A1BG_XM_001146669.7-1_9598     A1BG_XM_054464970.2-1_9600    95.556   
4790  A1BG_XM_001146669.7-1_9598     A1BG_XM_055239924.2-1_9590    95.556   
4791  A1BG_XM_001146669.7-1_9598  A1BG_XM_017509623.2-1_2715852    90.099   
4792  A1BG_XM_001146669.7-1_9598     A1BG_XM_031004187.3-1_9595    98.214   

      alignment_length  mismatches  gap_opens  q_start  q_end  s_start  s_e

In [ ]:
#get recirpocal best hits
best_hit=[]
for gene in blast['query'].unique() :
    df_gene = blast[blast['query']==gene]
    best_hit.append(df_gene.iloc[0])
best_df = pd.DataFrame(best_hit)
reci=[]
for index, row in best_df.iterrows():
    que = row['query']
    sub = row['subject']
    if que in best_df['subject'].values:
        match = best_df[best_df['subject']==que]
        if sub in match['query'].values:
            reci.append(row)
reci_best_hit = pd.DataFrame(reci)
print(reci_best_hit)
exons_list = []
length_list = []

with open("/home/collot/stage/collot/collot/out_ortho/fasta_homo_sapiens/A1BG_9606.fasta", "r") as f:
    for record in SeqIO.parse(f, "fasta"):
        exons_list.append(record.description)
        length_list.append(len(record.seq))

ali_length = []
orthologs = []
q_start = []
q_end = []
s_start = []
s_end = []
coverage = []
for exons in exons_list:
    if exons in reci_best_hit['query'].values:
        orthologs.append(reci_best_hit.loc[reci_best_hit['query']==exons, 'subject'].iloc[0])
        q_start.append(reci_best_hit.loc[reci_best_hit['query']==exons, 'q_start'].iloc[0])
        q_end.append(reci_best_hit.loc[reci_best_hit['query']==exons, 'q_end'].iloc[0])
        s_start.append(reci_best_hit.loc[reci_best_hit['query']==exons, 's_start'].iloc[0])
        s_end.append(reci_best_hit.loc[reci_best_hit['query']==exons, 's_end'].iloc[0])
        ali_length.append(reci_best_hit.loc[reci_best_hit['query']==exons, 'alignment_length'].iloc[0])
    elif exons in reci_best_hit['subject'].values:
        orthologs.append(reci_best_hit.loc[reci_best_hit['subject']==exons, 'query'].iloc[0])
        q_start.append(reci_best_hit.loc[reci_best_hit['subject']==exons, 'q_start'].iloc[0])
        q_end.append(reci_best_hit.loc[reci_best_hit['subject']==exons, 'q_end'].iloc[0])
        s_start.append(reci_best_hit.loc[reci_best_hit['subject']==exons, 's_start'].iloc[0])
        s_end.append(reci_best_hit.loc[reci_best_hit['subject']==exons, 's_end'].iloc[0])
        ali_length.append(reci_best_hit.loc[reci_best_hit['subject']==exons, 'alignment_length'].iloc[0])
    else:
        orthologs.append(None)
        q_start.append(0)
        q_end.append(0)
        s_start.append(0)
        s_end.append(0)
        ali_length.append(0)

df_all_exons = pd.DataFrame(list(zip(exons_list,length_list,ali_length, orthologs, q_start, q_end, s_start, s_end)), columns=['query_exons','query_length','alignment_length', 'best_hit','q_start', 'q_end', 's_start', 's_end' ])
df_all_exons['coverage']=(df_all_exons['alignment_length']/df_all_exons['query_length'])*100

for index, row in df_all_exons.iterrows():
    best_hit = row['best_hit']
    if best_hit is not None:

        match = df_all_exons.loc[df_all_exons['query_exons'] == best_hit, 'query_length']
        if not match.empty:
            df_all_exons.at[index, 'best_hit_length'] = match.iloc[0]
    else:
        df_all_exons.at[index, 'best_hit_length'] = None

print(df_all_exons)

                              query                      subject  identity  \
1           A1BG_NM_130786.4-8_9606  A1BG_XM_030819447.1-7_61853    97.059   
159         A1BG_NM_130786.4-1_9606   A1BG_XM_001146669.7-1_9598   100.000   
166   A1BG_XM_017509623.2-7_2715852   A1BG_XM_032263339.1-7_9515    98.864   
190   A1BG_XM_017509623.2-6_2715852   A1BG_XM_032263338.1-7_9515    99.653   
215   A1BG_XM_017509623.2-5_2715852   A1BG_XM_032263338.1-6_9515    98.316   
...                             ...                          ...       ...   
4699     A1BG_XM_001146669.7-5_9598   A1BG_XM_055103953.3-5_9597    99.663   
4727     A1BG_XM_001146669.7-4_9598   A1BG_XM_031004187.3-4_9595    98.535   
4754     A1BG_XM_001146669.7-3_9598   A1BG_XM_055103953.3-3_9597    99.259   
4783     A1BG_XM_001146669.7-2_9598   A1BG_XM_055103953.3-2_9597   100.000   
4788     A1BG_XM_001146669.7-1_9598      A1BG_NM_130786.4-1_9606   100.000   

      alignment_length  mismatches  gap_opens  q_start  q_end  